In [1]:
import weka.core.jvm as jvm
from weka.core.converters import Loader, Saver
from weka.core.dataset import Instances
from weka.filters import Filter

In [2]:
jvm.start()

DEBUG:weka.core.jvm:Adding bundled jars
DEBUG:weka.core.jvm:Classpath=['C:\\Users\\micha\\anaconda3\\envs\\weka_env\\Lib\\site-packages\\javabridge\\jars\\rhino-1.7R4.jar', 'C:\\Users\\micha\\anaconda3\\envs\\weka_env\\Lib\\site-packages\\javabridge\\jars\\runnablequeue.jar', 'C:\\Users\\micha\\anaconda3\\envs\\weka_env\\Lib\\site-packages\\javabridge\\jars\\cpython.jar', 'C:\\Users\\micha\\anaconda3\\envs\\weka_env\\lib\\site-packages\\weka\\lib\\arpack_combined.jar', 'C:\\Users\\micha\\anaconda3\\envs\\weka_env\\lib\\site-packages\\weka\\lib\\core.jar', 'C:\\Users\\micha\\anaconda3\\envs\\weka_env\\lib\\site-packages\\weka\\lib\\mtj.jar', 'C:\\Users\\micha\\anaconda3\\envs\\weka_env\\lib\\site-packages\\weka\\lib\\python-weka-wrapper.jar', 'C:\\Users\\micha\\anaconda3\\envs\\weka_env\\lib\\site-packages\\weka\\lib\\weka.jar']
DEBUG:weka.core.jvm:MaxHeapSize=default
DEBUG:weka.core.jvm:Package support disabled


In [3]:

loader = Loader(classname="weka.core.converters.ArffLoader")
data = loader.load_file("z_listy_7.arff")
data.class_is_last()
# -C num
# Choose attribute to be used for selection (default last).
# 
# -S num
# Numeric value to be used for selection on numeric attribute. Instances with values smaller than given value will be selected. (default 0)
# 
# -L index1,index2-index4,...
# Range of label indices to be used for selection on nominal attribute. First and last are valid indexes. (default all values)
# 
# -M
# Missing values count as a match. This setting is independent of the -V option. (default missing values don't match)
# 
# -V
# Invert matching sense.
# 
# -H
# When selecting on nominal attributes, removes header references to excluded values.


In [4]:
print("Number of instances before filtering(odmowa):", data.num_instances)
remove_filter = Filter(classname="weka.filters.unsupervised.instance.RemoveWithValues", options=[
    "-C", str(data.attribute_by_name("status pozyczki").index + 1),
    "-L", "2"])  # value to remove
remove_filter.inputformat(data)
data_after_odmowa = Instances.copy_instances(remove_filter.filter(data))

print("Number of instances after filtering(odmowa):", data_after_odmowa.num_instances)
print("Difference:", data.num_instances - data_after_odmowa.num_instances, "From csv info it should be 145")

Number of instances before filtering(odmowa): 1581
Number of instances after filtering(odmowa): 1436
Difference: 145 From csv info it should be 145
--------------------------------------------------


In [5]:
print("Number of instances before filtering(>900):", data_after_odmowa.num_instances)
remove_filter = Filter(classname="weka.filters.unsupervised.instance.RemoveWithValues", options=[
    "-C", str(data_after_odmowa.attribute_by_name("kwota kredytu").index + 1),
    "-S", "900",
    "-V"
])
remove_filter.inputformat(data_after_odmowa)

data_after_900 = Instances.copy_instances(remove_filter.filter(data_after_odmowa))
print("Number of instances after filtering(>900):", data_after_900.num_instances)

removed_instances = data_after_odmowa.num_instances - data_after_900.num_instances
print("Difference:", removed_instances, "From csv info it should be 62")

saver = Saver(classname="weka.core.converters.ArffSaver")
saver.save_file(data_after_900, "po_skrypcie_1.arff")

Number of instances before filtering(>900): 1436
Number of instances after filtering(>900): 1374
Difference: 62 From csv info it should be 62
--------------------------------------------------


In [6]:
remove_attribute = Filter(classname="weka.filters.unsupervised.attribute.Remove", options=["-R", str(data_after_900.attribute_by_name("status pozyczki").index + 1)])
remove_attribute.inputformat(data_after_900)
data_no_status_pozyczki = Instances.copy_instances(remove_attribute.filter(data_after_900))

saver = Saver(classname="weka.core.converters.ArffSaver")
saver.save_file(data_no_status_pozyczki, "po_skrypcie_2.arff")


In [7]:
jvm.stop()